In [82]:
import sys, os, cv2, glob
from skimage.measure import compare_ssim
import pandas as pd
import scipy.io as sio
import numpy as np
from skimage.measure import compare_mse as mse
from sklearn.metrics import mean_squared_error
from math import sqrt
#from vifvec import vifvec
##temporarily removing metric because pyPyrTools dependency is not supported on windows
##it is supported on linux tho!
from sewar import full_ref as fr

In [83]:
datapath_real = r'C:\Users\Garrett\Desktop\new_augmented_data\test'
datapath_fake = r'C:\Users\Garrett\Desktop\new_augmented_data\test_results_710_lr1_median_filter'
modelname = r'FINAL_EVAL_FILTERED_NORMAL'
outputpath = r'C:/Users/Garrett/Desktop/new_augmented_data/' 

results_savepath = os.path.join(os.path.abspath(outputpath),'FINAL_EVAL_metrics_results')
results_savename = os.path.join(results_savepath,(modelname+'.xlsx'))

print(results_savepath)
print(results_savename)

C:\Users\Garrett\Desktop\new_augmented_data\FINAL_EVAL_metrics_results
C:\Users\Garrett\Desktop\new_augmented_data\FINAL_EVAL_metrics_results\FINAL_EVAL_FILTERED_NORMAL.xlsx


In [84]:
classA = r'MTT'
classB = r'TTP'
classC = r'rCBF'
classD = r'rCBV'

In [85]:
# fix PSNR, fix RNMSE
# figure out what to do w PSNR afterwards

In [86]:
def make_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [87]:
def rmsdiff(im1, im2):
    # Calculate the root-mean-square difference between two images (already saved as matrices!)
    return sqrt(mse(im1,im2))

In [88]:
def allmetrics(img_1, img_2):
    compute_PSNR = cv2.PSNR(img_1, img_2)
    #compute_SSIM = compare_ssim(img_1, img_2)
    #compute_RNMSE = sqrt(mean_squared_error(make_gray(img_1), make_gray(img_2)))
    compute_RNMSE = rmsdiff(img_1, img_2)
    #compute_VIF = vifvec(img_1, img_2)
    #compute_VIFP = fr.vifp(img_1, img_2)
    #compute_UQI = fr.uqi(img_1, img_2)
    #compute_SCC = fr.scc(img_1, img_2)
    
    return (compute_PSNR, compute_RNMSE)

In [89]:
def allmetrics_orig(img_1, img_2):
    #compute_PSNR = cv2.PSNR(img_1, img_2)
    compute_SSIM = compare_ssim(img_1, img_2)
    #compute_RNMSE = sqrt(mean_squared_error(make_gray(img_1), make_gray(img_2)))
    #compute_RNMSE = rmsdiff(img_1, img_2)
    #compute_VIF = vifvec(img_1, img_2)
    #compute_VIFP = fr.vifp(img_1, img_2)
    compute_UQI = fr.uqi(img_1, img_2)
    #compute_SCC = fr.scc(img_1, img_2)
    
    return (compute_SSIM, compute_UQI)

In [90]:
num_patients = 0
patient_ids = []

metrics_classA = ['PSNR_'+classA,'SSIM_'+classA,'RMSE_'+classA,'UQI_'+classA]
metrics_classB = ['PSNR_'+classB,'SSIM_'+classB,'RMSE_'+classB,'UQI_'+classB]
metrics_classC = ['PSNR_'+classC,'SSIM_'+classC,'RMSE_'+classC,'UQI_'+classC]
metrics_classD = ['PSNR_'+classD,'SSIM_'+classD,'RMSE_'+classD,'UQI_'+classD]


#datapath = os.path.join(datapath,'test_latest','images')
datapath_contents = os.listdir(datapath_fake)
temp_list = [filename[:6] for filename in datapath_contents if filename[0]!='.' and len(filename)>=6]
[patient_ids.append(id) for id in temp_list if id not in patient_ids]

print(patient_ids)

results_A = pd.DataFrame(index = patient_ids, columns = metrics_classA)
results_B = pd.DataFrame(index = patient_ids, columns = metrics_classB)
results_C = pd.DataFrame(index = patient_ids, columns = metrics_classC)
results_D = pd.DataFrame(index = patient_ids, columns = metrics_classD)

print(results_A)

['000201', '000202', '000203', '000204', '000205', '000206', '000207', '000208', '000209', '000210', '000211', '000212', '000213', '000214', '000215', '000216', '000217', '000218', '000219', '000220', '000221', '000222', '000223', '000227', '000239', '000246', '000271', '000272', '000281', '000301', '000306', '000318', '000324', '000351', '000373', '000376', '000397', '000405', '000408', '000414', '000416', '000423', '000438', '000455', '000481', '000487', '000517', '000528', '000534', '000549', '000552', '000578', '000580', '000583', '000593', '000597', '000609', '000614', '000622', '000650', '000651', '000680']
       PSNR_MTT SSIM_MTT RMSE_MTT UQI_MTT
000201      NaN      NaN      NaN     NaN
000202      NaN      NaN      NaN     NaN
000203      NaN      NaN      NaN     NaN
000204      NaN      NaN      NaN     NaN
000205      NaN      NaN      NaN     NaN
...         ...      ...      ...     ...
000614      NaN      NaN      NaN     NaN
000622      NaN      NaN      NaN     NaN
0

In [91]:
for pid in patient_ids:
    print(pid)
    ppath = os.path.join(datapath_fake,(pid+'*output'+'*.png'))
    pfiles = glob.glob(ppath)
    
    pPSNR_A = 0.00
    pSSIM_A = 0.00
    pRNMSE_A = 0.00
    #pVIFP_A = 0.00
    pUQI_A = 0.00
    #pSCC_A = 0.00

    pPSNR_B = 0.00
    pSSIM_B = 0.00
    pRNMSE_B = 0.00
    #pVIFP_B = 0.00
    pUQI_B = 0.00
    #pSCC_B = 0.00

    pPSNR_C = 0.00
    pSSIM_C = 0.00
    pRNMSE_C = 0.00
    #pVIFP_C = 0.00
    pUQI_C = 0.00
    #pSCC_C = 0.00

    pPSNR_D = 0.00
    pSSIM_D = 0.00
    pRNMSE_D = 0.00
    #pVIFP_D = 0.00
    pUQI_D = 0.00
    #pSCC_D = 0.00
    #print(len(pfiles))
    
    for file in pfiles:
        path_orig, file_fake = os.path.split(file)
        file_real = file_fake.replace('_output','').replace('png','bmp')
        #file_real = file_fake.replace('_output','')
        
        img_fake = cv2.imread(os.path.join(datapath_fake,file_fake))
        img_real = cv2.imread(os.path.join(datapath_real,file_real))
        
        unit = int(img_fake.shape[1] / 4)
        #print(unit)
        img_fake_A_u = make_gray(img_fake[:,0:unit,:])
        img_fake_B_u = make_gray(img_fake[:,unit:unit*2,:])
        img_fake_C_u = make_gray(img_fake[:,unit*2:unit*3,:])
        img_fake_D_u = make_gray(img_fake[:,unit*3:,:])
        
        # don't start at 0 because first one is NCCT
        img_real_A_u = make_gray(img_real[:,unit:unit*2,:])
        img_real_B_u = make_gray(img_real[:,unit*2:unit*3,:])
        img_real_C_u = make_gray(img_real[:,unit*3:unit*4,:])
        img_real_D_u = make_gray(img_real[:,unit*4:,:])
        
        x_a,y_a,w_a,h_a = cv2.boundingRect(img_real_A_u)
        x_b,y_b,w_b,h_b = cv2.boundingRect(img_real_B_u)
        x_c,y_c,w_c,h_c = cv2.boundingRect(img_real_C_u)
        x_d,y_d,w_d,h_d = cv2.boundingRect(img_real_D_u)
        
        img_real_A_orig = img_real_A_u[y_a:y_a+h_a, x_a:x_a+w_a].astype('float')
        img_real_A = cv2.normalize(img_real_A_orig, None, 0.0, 20.0, cv2.NORM_MINMAX)
        img_fake_A_orig = img_fake_A_u[y_a:y_a+h_a, x_a:x_a+w_a].astype('float')
        img_fake_A = cv2.normalize(img_fake_A_orig, None, 0.0, 20.0, cv2.NORM_MINMAX)
        img_real_B_orig = img_real_B_u[y_b:y_b+h_b, x_b:x_b+w_b].astype('float')
        img_real_B = cv2.normalize(img_real_B_orig, None, 0.0, 12.0, cv2.NORM_MINMAX)
        img_fake_B_orig = img_fake_B_u[y_b:y_b+h_b, x_b:x_b+w_b].astype('float')
        img_fake_B = cv2.normalize(img_fake_B_orig, None, 0.0, 12.0, cv2.NORM_MINMAX)
        img_real_C_orig = img_real_C_u[y_c:y_c+h_c, x_c:x_c+w_c].astype('float')
        img_real_C = cv2.normalize(img_real_C_orig, None, 0.0, 60.0, cv2.NORM_MINMAX)
        img_fake_C_orig = img_fake_C_u[y_c:y_c+h_c, x_c:x_c+w_c].astype('float')
        img_fake_C = cv2.normalize(img_fake_C_orig, None, 0.0, 60.0, cv2.NORM_MINMAX)
        img_real_D_orig = img_real_D_u[y_d:y_d+h_d, x_d:x_d+w_d].astype('float')
        img_real_D = cv2.normalize(img_real_D_orig, None, 0.0, 4.0, cv2.NORM_MINMAX)
        img_fake_D_orig = img_fake_D_u[y_d:y_d+h_d, x_d:x_d+w_d].astype('float')
        img_fake_D = cv2.normalize(img_fake_D_orig, None, 0.0, 4.0, cv2.NORM_MINMAX)
        
        
        """
        print(img_fake_A.shape)
        print(img_fake_B.shape)
        print(img_fake_C.shape)
        print(img_fake_D.shape)
        print(img_real_A.shape)
        print(img_real_B.shape)
        print(img_real_C.shape)
        print(img_real_D.shape)
        
        
        #PSNR, SSIM, RNMSE, VIF, VIFP, UQI, SCC
        compute_PSNR = cv2.PSNR(img_1, img_2)
        compute_SSIM = compare_ssim(make_gray(img_1), make_gray(img_2))
        #compute_RNMSE = sqrt(mean_squared_error(make_gray(img_1), make_gray(img_2)))
        compute_RNMSE = rmsdiff(img_1, img_2)
        #compute_VIF = vifvec(img_1, img_2)
        compute_VIFP = fr.vifp(img_1, img_2)
        compute_UQI = fr.uqi(img_1, img_2)
        compute_SCC = fr.scc(img_1, img_2)
        """
        """
        img_fake_A_u[img_fake_A_u==0]=np.nan
        img_fake_B_u[img_fake_B_u==0]=np.nan
        img_fake_C_u[img_fake_C_u==0]=np.nan
        img_fake_D_u[img_fake_D_u==0]=np.nan
        img_real_A_u[img_real_A_u==0]=np.nan
        img_real_B_u[img_real_B_u==0]=np.nan
        img_real_C_u[img_real_C_u==0]=np.nan
        img_real_D_u[img_real_D_u==0]=np.nan
        """
        """
        (compute_PSNR_A, compute_RNMSE_A) = allmetrics(img_fake_A, img_real_A)
        (compute_PSNR_B, compute_RNMSE_B) = allmetrics(img_fake_B, img_real_B)
        (compute_PSNR_C, compute_RNMSE_C) = allmetrics(img_fake_C, img_real_C)
        (compute_PSNR_D, compute_RNMSE_D) = allmetrics(img_fake_D, img_real_D)
        """
        
        (compute_PSNR_A, compute_RNMSE_A) = allmetrics(img_fake_A_orig, img_real_A_orig)
        (compute_PSNR_B, compute_RNMSE_B) = allmetrics(img_fake_B_orig, img_real_B_orig)
        (compute_PSNR_C, compute_RNMSE_C) = allmetrics(img_fake_C_orig, img_real_C_orig)
        (compute_PSNR_D, compute_RNMSE_D) = allmetrics(img_fake_D_orig, img_real_D_orig)
        
        """
        (compute_SSIM_A, compute_UQI_A) = allmetrics_orig(img_fake_A_orig, img_real_A_orig)
        (compute_SSIM_B, compute_UQI_B) = allmetrics_orig(img_fake_B_orig, img_real_B_orig)
        (compute_SSIM_C, compute_UQI_C) = allmetrics_orig(img_fake_C_orig, img_real_C_orig)
        (compute_SSIM_D, compute_UQI_D) = allmetrics_orig(img_fake_D_orig, img_real_D_orig)
        """
        #best results
        
        #(compute_PSNR_A, compute_RNMSE_A) = allmetrics(img_fake_A, img_real_A)
        #(compute_PSNR_B, compute_RNMSE_B) = allmetrics(img_fake_B, img_real_B)
        #(compute_PSNR_C, compute_RNMSE_C) = allmetrics(img_fake_C, img_real_C)
        #(compute_PSNR_D, compute_RNMSE_D) = allmetrics(img_fake_D, img_real_D)
        
        
        (compute_SSIM_A, compute_UQI_A) = allmetrics_orig(img_fake_A_u, img_real_A_u)
        (compute_SSIM_B, compute_UQI_B) = allmetrics_orig(img_fake_B_u, img_real_B_u)
        (compute_SSIM_C, compute_UQI_C) = allmetrics_orig(img_fake_C_u, img_real_C_u)
        (compute_SSIM_D, compute_UQI_D) = allmetrics_orig(img_fake_D_u, img_real_D_u)
    
        pPSNR_A += compute_PSNR_A
        pSSIM_A += compute_SSIM_A
        pRNMSE_A += compute_RNMSE_A
        #pVIF += compute_VIF
        #pVIFP_A += compute_VIFP_A
        pUQI_A += compute_UQI_A
        #pSCC_A += compute_SCC_A

        pPSNR_B += compute_PSNR_B
        pSSIM_B += compute_SSIM_B
        pRNMSE_B += compute_RNMSE_B
        #pVIF += compute_VIF
        #pVIFP_B += compute_VIFP_B
        pUQI_B += compute_UQI_B
        #pSCC_B += compute_SCC_B
        
        pPSNR_C += compute_PSNR_C
        pSSIM_C += compute_SSIM_C
        pRNMSE_C += compute_RNMSE_C
        #pVIF += compute_VIF
        #pVIFP_C += compute_VIFP_C
        pUQI_C += compute_UQI_C
        #pSCC_C += compute_SCC_C

        pPSNR_D += compute_PSNR_D
        pSSIM_D += compute_SSIM_D
        pRNMSE_D += compute_RNMSE_D
        #pVIF += compute_VIF
        #pVIFP_D += compute_VIFP_D
        pUQI_D += compute_UQI_D
        #pSCC_D += compute_SCC_D
        
    pPSNR_A = pPSNR_A / len(pfiles)
    pSSIM_A = pSSIM_A / len(pfiles)
    pRNMSE_A = pRNMSE_A / len(pfiles)
    #pVIF_A = pVIF_A / len(pfiles)
    #pVIFP_A = pVIFP_A / len(pfiles)
    pUQI_A = pUQI_A / len(pfiles)
    #pSCC_A = pSCC_A / len(pfiles)

    pPSNR_B = pPSNR_B / len(pfiles)
    pSSIM_B = pSSIM_B / len(pfiles)
    pRNMSE_B = pRNMSE_B / len(pfiles)
    #pVIF_B = pVIF_B / len(pfiles)
    #pVIFP_B = pVIFP_B / len(pfiles)
    pUQI_B = pUQI_B / len(pfiles)
    #pSCC_B = pSCC_B / len(pfiles)

    pPSNR_C = pPSNR_C / len(pfiles)
    pSSIM_C = pSSIM_C / len(pfiles)
    pRNMSE_C = pRNMSE_C / len(pfiles)
    #pVIF_C = pVIF_C / len(pfiles)
    #pVIFP_C = pVIFP_C / len(pfiles)
    pUQI_C = pUQI_C / len(pfiles)
    #pSCC_C = pSCC_C / len(pfiles)

    pPSNR_D = pPSNR_D / len(pfiles)
    pSSIM_D = pSSIM_D / len(pfiles)
    pRNMSE_D = pRNMSE_D / len(pfiles)
    #pVIF_D = pVIF_D / len(pfiles)
    #pVIFP_D = pVIFP_D / len(pfiles)
    pUQI_D = pUQI_D / len(pfiles)
    #pSCC_D = pSCC_D / len(pfiles)
    
    results_A['PSNR_'+classA][pid] = pPSNR_A
    results_A['SSIM_'+classA][pid] = pSSIM_A
    results_A['RMSE_'+classA][pid] = pRNMSE_A
    #results['VIF_'+classA][pid] = pVIF_A
    #results_A['VIFP_'+classA][pid] = pVIFP_A
    results_A['UQI_'+classA][pid] = pUQI_A
    #results_A['SCC_'+classA][pid] = pSCC_A

    results_B['PSNR_'+classB][pid] = pPSNR_B
    results_B['SSIM_'+classB][pid] = pSSIM_B
    results_B['RMSE_'+classB][pid] = pRNMSE_B
    #results['VIF_'+classB][pid] = pVIF_B
    #results_B['VIFP_'+classB][pid] = pVIFP_B
    results_B['UQI_'+classB][pid] = pUQI_B
    #results_B['SCC_'+classB][pid] = pSCC_B

    results_C['PSNR_'+classC][pid] = pPSNR_C
    results_C['SSIM_'+classC][pid] = pSSIM_C
    results_C['RMSE_'+classC][pid] = pRNMSE_C
    #results['VIF_'+classC][pid] = pVIF_C
    #results_C['VIFP_'+classC][pid] = pVIFP_C
    results_C['UQI_'+classC][pid] = pUQI_C
    #results_C['SCC_'+classC][pid] = pSCC_C
    
    results_D['PSNR_'+classD][pid] = pPSNR_D
    results_D['SSIM_'+classD][pid] = pSSIM_D
    results_D['RMSE_'+classD][pid] = pRNMSE_D
    #results['VIF_'+classD][pid] = pVIF_D
    #results_D['VIFP_'+classD][pid] = pVIFP_D
    results_D['UQI_'+classD][pid] = pUQI_D
    #results_D['SCC_'+classD][pid] = pSCC_D

    print(f'Done with patient {pid}')
    print(results_A)
    print(results_B)
    print(results_C)
    print(results_D)
    

000201


C:\Users\Garrett\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_mse has been moved to skimage.metrics.mean_squared_error. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Garrett\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until


Done with patient 000201
       PSNR_MTT  SSIM_MTT RMSE_MTT   UQI_MTT
000201  16.7594  0.805525  37.4031  0.965125
000202      NaN       NaN      NaN       NaN
000203      NaN       NaN      NaN       NaN
000204      NaN       NaN      NaN       NaN
000205      NaN       NaN      NaN       NaN
...         ...       ...      ...       ...
000614      NaN       NaN      NaN       NaN
000622      NaN       NaN      NaN       NaN
000650      NaN       NaN      NaN       NaN
000651      NaN       NaN      NaN       NaN
000680      NaN       NaN      NaN       NaN

[62 rows x 4 columns]
       PSNR_TTP  SSIM_TTP RMSE_TTP   UQI_TTP
000201  16.0936  0.802057  41.3859  0.926192
000202      NaN       NaN      NaN       NaN
000203      NaN       NaN      NaN       NaN
000204      NaN       NaN      NaN       NaN
000205      NaN       NaN      NaN       NaN
...         ...       ...      ...       ...
000614      NaN       NaN      NaN       NaN
000622      NaN       NaN      NaN       NaN
000650 

C:\Users\Garrett\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_mse has been moved to skimage.metrics.mean_squared_error. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Garrett\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until


Done with patient 000202
       PSNR_MTT  SSIM_MTT RMSE_MTT   UQI_MTT
000201  16.7594  0.805525  37.4031  0.965125
000202   16.327  0.783903  39.5172   0.95546
000203      NaN       NaN      NaN       NaN
000204      NaN       NaN      NaN       NaN
000205      NaN       NaN      NaN       NaN
...         ...       ...      ...       ...
000614      NaN       NaN      NaN       NaN
000622      NaN       NaN      NaN       NaN
000650      NaN       NaN      NaN       NaN
000651      NaN       NaN      NaN       NaN
000680      NaN       NaN      NaN       NaN

[62 rows x 4 columns]
       PSNR_TTP  SSIM_TTP RMSE_TTP   UQI_TTP
000201  16.0936  0.802057  41.3859  0.926192
000202   16.492  0.790716  39.4171  0.907806
000203      NaN       NaN      NaN       NaN
000204      NaN       NaN      NaN       NaN
000205      NaN       NaN      NaN       NaN
...         ...       ...      ...       ...
000614      NaN       NaN      NaN       NaN
000622      NaN       NaN      NaN       NaN
000650 

...

Done with patient 000650
       PSNR_MTT  SSIM_MTT RMSE_MTT   UQI_MTT
000201  16.7594  0.805525  37.4031  0.965125
000202   16.327  0.783903  39.5172   0.95546
000203  17.3749  0.812746  34.6954  0.963746
000204  14.3977  0.765207  48.8312  0.916222
000205  16.1687  0.786387  39.6729  0.946513
...         ...       ...      ...       ...
000614  14.7865  0.811976  46.8236  0.955848
000622  14.5346  0.827925  48.0013  0.946695
000650  15.2339  0.819523  44.8237  0.953192
000651      NaN       NaN      NaN       NaN
000680      NaN       NaN      NaN       NaN

[62 rows x 4 columns]
       PSNR_TTP  SSIM_TTP RMSE_TTP   UQI_TTP
000201  16.0936  0.802057  41.3859  0.926192
000202   16.492  0.790716  39.4171  0.907806
000203  19.8515  0.824139  26.1547  0.933164
000204  15.5809  0.770463  43.7822  0.842014
000205  18.9605  0.817985   29.166  0.930463
...         ...       ...      ...       ...
000614  13.9315  0.817681   54.569  0.914866
000622  15.5633  0.823706  43.3562  0.906701
000650 

C:\Users\Garrett\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_mse has been moved to skimage.metrics.mean_squared_error. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Garrett\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until


Done with patient 000651
       PSNR_MTT  SSIM_MTT RMSE_MTT   UQI_MTT
000201  16.7594  0.805525  37.4031  0.965125
000202   16.327  0.783903  39.5172   0.95546
000203  17.3749  0.812746  34.6954  0.963746
000204  14.3977  0.765207  48.8312  0.916222
000205  16.1687  0.786387  39.6729  0.946513
...         ...       ...      ...       ...
000614  14.7865  0.811976  46.8236  0.955848
000622  14.5346  0.827925  48.0013  0.946695
000650  15.2339  0.819523  44.8237  0.953192
000651  14.6261  0.818817   47.606  0.956536
000680      NaN       NaN      NaN       NaN

[62 rows x 4 columns]
       PSNR_TTP  SSIM_TTP RMSE_TTP   UQI_TTP
000201  16.0936  0.802057  41.3859  0.926192
000202   16.492  0.790716  39.4171  0.907806
000203  19.8515  0.824139  26.1547  0.933164
000204  15.5809  0.770463  43.7822  0.842014
000205  18.9605  0.817985   29.166  0.930463
...         ...       ...      ...       ...
000614  13.9315  0.817681   54.569  0.914866
000622  15.5633  0.823706  43.3562  0.906701
000650 

C:\Users\Garrett\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_mse has been moved to skimage.metrics.mean_squared_error. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Garrett\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until


Done with patient 000680
       PSNR_MTT  SSIM_MTT RMSE_MTT   UQI_MTT
000201  16.7594  0.805525  37.4031  0.965125
000202   16.327  0.783903  39.5172   0.95546
000203  17.3749  0.812746  34.6954  0.963746
000204  14.3977  0.765207  48.8312  0.916222
000205  16.1687  0.786387  39.6729  0.946513
...         ...       ...      ...       ...
000614  14.7865  0.811976  46.8236  0.955848
000622  14.5346  0.827925  48.0013  0.946695
000650  15.2339  0.819523  44.8237  0.953192
000651  14.6261  0.818817   47.606  0.956536
000680  13.8977  0.763347  51.7709  0.930826

[62 rows x 4 columns]
       PSNR_TTP  SSIM_TTP RMSE_TTP   UQI_TTP
000201  16.0936  0.802057  41.3859  0.926192
000202   16.492  0.790716  39.4171  0.907806
000203  19.8515  0.824139  26.1547  0.933164
000204  15.5809  0.770463  43.7822  0.842014
000205  18.9605  0.817985   29.166  0.930463
...         ...       ...      ...       ...
000614  13.9315  0.817681   54.569  0.914866
000622  15.5633  0.823706  43.3562  0.906701
000650 

In [92]:
if not os.path.isdir(results_savepath):
    if not os.path.isdir(outputpath):
        os.mkdir(outputpath)
        
    os.mkdir(results_savepath)
    
with pd.ExcelWriter(results_savename) as writer:
    results_A.to_excel(writer, sheet_name=classA)
    results_B.to_excel(writer, sheet_name=classB)
    results_C.to_excel(writer, sheet_name=classC)
    results_D.to_excel(writer, sheet_name=classD)

print('Done saving results!')

Done saving results!
